In [1]:
import twstock

In [2]:
stock = twstock.Stock('2330') #取得該支股票歷史價格

In [3]:
latest_20 = stock.price[-20:]

In [4]:
print(stock.moving_average(latest_20, 5))
print(stock.moving_average(latest_20, 10))
print(stock.moving_average(latest_20, 20))

[239.1, 239.3, 239.1, 239.7, 240.6, 241.5, 242.0, 241.7, 239.9, 238.4, 235.1, 232.5, 231.9, 231.0, 229.5, 229.6]
[240.3, 240.65, 240.4, 239.8, 239.5, 238.3, 237.25, 236.8, 235.45, 233.95, 232.35]
[236.32]


In [7]:
# 取得今日5, 10, 20的移動平均
def getCurrentMA(latest_20):
    mv_5_3d = stock.moving_average(latest_20, 5)[-3:]
    mv_10_3d = stock.moving_average(latest_20, 10)[-3:]
    mv_20_1d = stock.moving_average(latest_20, 20)[-1]
    
    return (mv_5_3d, mv_10_3d, mv_20_1d)

getCurrentMA(latest_20)

([231.0, 229.5, 229.6], [235.45, 233.95, 232.35], 236.32)

In [23]:
'''
檢查糾結
input: MA list, 三日的MA差距
'''

def isSticky(mv_list, cond_1 = 3, cond_2 = 4, cond_3 = 5):
    first = abs(mv_list[2] - mv_list[1][-1]) # 20MA - 10MA
    second = abs(mv_list[1][-2] - mv_list[0][-2]) # 10MA - 5MA
    third = abs(mv_list[1][-3] - mv_list[0][-3]) # 10MA - 5MA
    
    return first <= cond_1 and second <= cond_2 and third <= cond_3
    
isSticky(getCurrentMA(latest_20), 10, 10, 10)

True

In [22]:
'''
K棒大小: close/open -1
'''

def isKsize(close, open_p, size = 5):
    return ((close/open_p) - 1)*100 >= size
    
isKsize(100, 98)

False

In [21]:
'''
高價與收盤價距離
'''

def is_HC_range(close, high, dis = 3):
    return high - close <= dis
    
    
is_HC_range(20, 20)

True

In [24]:
'''
收盤大於幾倍5MA
'''

def is_close_big_than_5MA(close, today_5MA, dis = 1):
    return close / today_5MA >= dis

is_close_big_than_5MA(200, 198)

True

In [26]:
'''
成交量大小範圍
'''

def isVolumn(volumn, dis = (1000,)):
    if len(dis) == 1: return volumn >= dis[0]
    
    return volumn >= dis[0] and volumn <= dis[1]

print(isVolumn(1001, (1000,)))
print(isVolumn(555, (560, 600)))

True
False


In [57]:
'''
檢查有無達成均線糾結向上突破條件
'''
class BreakChecker:
    def __init__(self, code):
        self.stock = twstock.Stock(code)
        self.latest_20 = self.stock.price[-20:]
        self.today_high = self.stock.high[-1]
        self.today_close = self.stock.price[-1]
        self.today_open = self.stock.open[-1]
        self.volumn = self.stock.capacity[-1] / 1000
        
        self.ma_list = self.getCurrentMA(self.latest_20)
        
        
    # 取得今日5, 10, 20的移動平均
    def getCurrentMA(self, latest_20):
        mv_5_3d = stock.moving_average(latest_20, 5)[-3:]
        mv_10_3d = stock.moving_average(latest_20, 10)[-3:]
        mv_20_1d = stock.moving_average(latest_20, 20)[-1]

        return (mv_5_3d, mv_10_3d, mv_20_1d)

    '''
    檢查糾結
    input: MA list, 三日的MA差距
    '''
    def isSticky(self, ma_list, cond_1 = 3, cond_2 = 4, cond_3 = 5):
        first = abs(ma_list[2] - ma_list[1][-1]) # 20MA - 10MA
        second = abs(ma_list[1][-2] - ma_list[0][-2]) # 10MA - 5MA
        third = abs(ma_list[1][-3] - ma_list[0][-3]) # 10MA - 5MA

        return first <= cond_1 and second <= cond_2 and third <= cond_3
    
    '''
    K棒大小: close/open -1
    '''
    def isKsize(self, close, open_p, size = 1):
        return ((close/open_p) - 1)*100 >= size
    
    '''
    高價與收盤價距離
    '''
    def is_HC_range(self, close, high, dis = 5):
        return high - close <= dis
    
    '''
    收盤大於幾倍5MA
    '''
    def is_close_big_than_5MA(self, close, today_5MA, dis = 1):
        return close / today_5MA >= dis
    
    '''
    成交量大小範圍
    '''
    def isVolumn(self, volumn, dis = (500, 1500)):
        if len(dis) == 1: return volumn >= dis[0]

        return volumn >= dis[0] and volumn <= dis[1]
    
    '''
    主要檢查method
    '''
    def isBreak(self):
        return (self.isSticky(self.ma_list) and 
                self.isKsize(self.today_close, self.today_open) and
                self.is_HC_range(self.today_close, self.today_high) and
                self.is_close_big_than_5MA(self.today_close, self.ma_list[0][-1]))

In [62]:
breaker = BreakChecker('2634')
breaker.isBreak()

True